In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
from basic.external import cpa_parse, get_yandex_data
from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
import csv, sqlite3
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks

# sqlite_file = 'temp/db2.sqlite' # name of the sqlite database file
# conn = sqlite3.connect(sqlite_file)

client, es = connect()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [2]:
# !pip install pickledb pymongo elasticsearch==6.8 oauth2client gspread df2gspread \
# robobrowser beautifulsoup4 lxml yourls Werkzeug==0.16.1 paramiko xlsxwriter pyminizip \
# html5lib zodiac_sign python-pptx plotly df2gspread gspread

In [3]:
def get_kibana_registrations(es, since, till):
    chunk = 1000
    print('get_kibana_registrations')    
    gte = (str((datetime.combine(since, datetime.min.time())-timedelta(seconds=3*60*60)).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": chunk,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
    t={}
    for i in range(0,len(hits)):
        t = hits[i]['_source']
        t.update(t['params'])
        del t['params']
        res.append(t)
    delk = pd.DataFrame(res)
    delk['dt'] = delk['timestamp'].apply(lambda x: (datetime.strptime(x[:x.rfind(':')+3], 
                                                                      '%Y-%m-%dT%H:%M:%S')+timedelta(hours=3)).date())
    if len(df) > 0:
        df['date'] = df['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return df

In [4]:
def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
           "status": {"$nin": ["NEW", "ERROR"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)

    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1, 'clickId':1,
                                                                "updatedAt":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
#     df['product_type'] = df['product'].apply(lambda x: x['productType'])
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])

    return df

In [5]:
def get_exact_claims2(since, till, client, userIds):
    req = {
        "createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
           "_id": {"$in": [ObjectId(product) for ids in ids]},
           "borrower.userId": {"$in": userIds},
           "status": {"$nin": ["NEW", "ERROR"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 1000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['product_type'] = df['product'].apply(lambda x: x['productType'])
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['product_id'] = df['product'].apply(lambda x: x['_id'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])

    return df

In [6]:
since = date(2021,9,1)
till = date(2021,10, 1)
du = get_users(client)

183399;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;mainFields done; lastName done; firstName done; middleName done; passportIssueDate done; passportIssuedBy done; registrationAddress done; addressOfResidence done; registrationAddress.street done; addressOfResidence.street done; passportNumber done; dateOfBirth done; registrationAddress.city done; registrationAddress.region.kladr done; placeOfBirth done; addressOfResidence.city done; addressOfResidence.region.kladr done; subdivisionCode done; salary done; amount done; gender done; loanPeriod done; seniorityLatest done; typeOfEmployment done; 

In [7]:
len(du), len(du[du['dateOfBirth'].notnull()])

(183393, 161941)

In [8]:
# du['obl'] = du['subdivisionCode'].apply(lambda x: x[:2] if pd.notnull(x) else np.nan)
# du['obl']

In [9]:
# du['obl'] = du['obl'].apply(lambda x: '1' if 
#                                   x in ['77','50','78','47','66',
#                                             '34','35','52','53','61','63','02'] else x)

In [34]:
dff_all = get_exact_claims(date.today()-timedelta(days=10), date.today(), client)
#dff_all = get_exact_claims(client)

dff_all['userId'] = dff_all['borrower'].apply(lambda x: x['userId'])
dff_all['phone'] = dff_all['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)
dff_all['name'] = dff_all['borrower'].apply(lambda x: x['fullName'].strip() if 'fullName' in x.keys() else np.nan)
dff_all['name'].apply(lambda x: x[x.find(' '):x.rfind(' ')] if pd.notnull(x) else np.nan)

450


0                NaN
1                NaN
2           Вячеслав
3            Эльмира
4         Витальевич
5           Нинанина
6             Марина
7              Мария
8             Марина
9            Алексей
10            Карлен
11            Сергей
12            Карлен
13          Кристина
14          анатолий
15               NaN
16           Евгений
17             Алина
18            Михаил
19             Роман
20         Александр
21         Анастасия
22            Мариам
23            Мариам
24             Барно
25          Владимир
26            Михаил
27         Анастасия
28           Наталья
29         Анастасия
30                  
31             Дарья
32          Светлана
33              Юрий
34         Александр
35           Дмитрий
36             Елена
37           Дмитрий
38            Оксана
39          Мирослав
40            Салман
41         Александр
42            Залина
43           Эльмира
44         Александр
45           Наталья
46            Сергей
47           

In [35]:
# du['dateOfBirth1']= pd.to_datetime(du['dateOfBirth'])
# du['date_today'] = pd.to_datetime(date.today())

In [36]:
# du['yearOfBirth2'] = (du['dateOfBirth1'].dt.year)
# du['year_today'] = (du['date_today'].dt.year).astype('float')

In [ ]:
# du['age'] = (du['year_today']-du['yearOfBirth2'])

In [ ]:
# du['age'].unique()

In [37]:
# #Разобъем всех id по возрасту на 4 группы : 'до 30', '30-45', '45-60', '60+' 
# def income_group(age):
#     if age <= 24: return '24'
#     if 24 < age <= 34: return '25-34'
#     if 34 < age <= 44: return '35-44'
#     if 44 < age <=54: return '45-54'
#     if 54 < age <=64: return '55-64'
#     if 64 < age: return '65+'
    


#     else: 
#         return 'NotBirthDay'
    
# du['age_group'] = du['age'].apply(income_group)
# du['age_group'].value_counts()

In [38]:
dff_all.head()

,_id,clickId,type,createdAt,updatedAt,status,borrower,partner,product,statusFields,adapterName,id,dt,product_name,userId,phone,name
0,61b280d4f81409000159675e,766d372d-36db-42a0-9e50-d75b666211a4,LINK,2021-12-09 22:19:00.221,2021-12-10 04:15:15.345,ACCEPTED,"{'_id': None, 'userId': '61b280c6e6086b00019e42d3'}",banki.ru,"{'_id': 6087fee0b6d00a0001344eba, 'name': 'Одобрим.ру + Banki.ru= автокредит для вас', 'productType': 'AUTOCREDIT', 'productName': 'Автокредит'}","{'NOTE': 'OfferId not found', 'LINK': None}",NaN,61b280d4f81409000159675e,2021-12-10,Одобрим.ру + Banki.ru= автокредит для вас,61b280c6e6086b00019e42d3,NaN,NaN
1,61b285d6f814090001596770,NaN,LINK,2021-12-09 22:40:22.789,2021-12-09 22:56:03.035,REJECTED,"{'_id': None, 'userId': '61b285d6eae5be0001fb7b69'}",еКапуста,"{'_id': 5abb69e421f7ea00074349c9, 'name': 'еКапуста', 'productType': 'LOAN', 'productName': 'Займ'}","{'NOTE': 'OfferId not found', 'LINK': None}",,61b285d6f814090001596770,2021-12-10,еКапуста,61b285d6eae5be0001fb7b69,NaN,NaN
2,61b2b506435732000151e5cd,0a0ceda6-9521-4d1b-a77a-38a68be0d45f,LINK,2021-12-10 02:01:42.740,2021-12-10 03:07:07.690,REJECTED,"{'_id': None, 'userId': '61b2b351e6086b00019e5b69', 'fullName': 'Клеценко Вячеслав Николаевич', 'email': 'kletsenko87@gmail.com', 'phone': '79231289073'}",Доброзайм,"{'_id': 5d4058712a86100001f5ba35, 'name': 'Доброзайм', 'productType': 'LOAN', 'productName': 'Займ'}","{'NOTE': 'OfferId not found', 'LINK': None}",LEADGID_PARTNER,61b2b506435732000151e5cd,2021-12-10,Доброзайм,61b2b351e6086b00019e5b69,79231289073,Клеценко Вячеслав Николаевич
3,61b2b6a5128c070001900260,34f85956-3ea5-4fcf-bfd9-9c4c0616e5e0,LINK,2021-12-10 02:08:37.314,2021-12-10 02:18:54.096,REJECTED,"{'_id': None, 'userId': '61b2a262eae5be0001fb8888', 'fullName': 'Цаганова Эльмира Лубсановна', 'email': 'budashap0318@gmail.com', 'phone': '79025311178'}",МТС Банк,"{'_id': 5ca741818c85df000167ad3f, 'name': 'Кредит наличными ', 'productType': 'CASH_LOAN', 'productName': 'Кредит наличными'}","{'NOTE': '4394', 'LINK': None}",LEADGID_PARTNER,61b2b6a5128c070001900260,2021-12-10,Кредит наличными,61b2a262eae5be0001fb8888,79025311178,Цаганова Эльмира Лубсановна
4,61b2b891435732000151e5f9,4a3ff16b-eafb-4666-8314-ea06e0ab5bb2,LINK,2021-12-10 02:16:49.223,2021-12-10 02:21:02.265,ACCEPTED,"{'_id': None, 'userId': '61b2b7b4e6086b00019e5d78', 'fullName': 'Максим Витальевич Бабенко', 'email': 'babenko97.maks97@mail.ru', 'phone': '79000973431'}",СМСФинанс,"{'_id': 5abb6ef821f7ea00074349d6, 'name': 'СМСФинанс', 'productType': 'LOAN', 'productName': 'Займ'}","{'LINK': None, 'NOTE': 'OfferId not found'}",,61b2b891435732000151e5f9,2021-12-10,СМСФинанс,61b2b7b4e6086b00019e5d78,79000973431,Максим Витальевич Бабенко


In [13]:
dff_all['productType'] = dff_all['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else 'other')

In [14]:
dff2 = dff_all[dff_all['status'].isin(['APPROVED', 'ISSUED'])].groupby('userId', as_index=False).agg({ 'id': 'count',
                                'productType': 'first'})

In [15]:
dff = dff_all.groupby('userId', as_index=False).agg({ 'id': 'count',
                                'productType': 'first'})

In [16]:
def cre_get(es):
    
    chunk = 5000
    gte = (str(datetime.combine(date(2020,2,1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", ascending=False).drop_duplicates('userId', keep='first')
      
            
    return df[['userId', 'score']].reset_index(drop=True)

In [17]:
cre = cre_get(es)

231938; 226938; 221938; 216938; 211938; 206938; 201938; 196938; 191938; 186938; 181938; 176938; 171938; 166938; 161938; 156938; 151938; 146938; 141938; 136938; 131938; 126938; 121938; 116938; 111938; 106938; 101938; 96938; 91938; 86938; 81938; 76938; 71938; 66938; 61938; 56938; 51938; 46938; 41938; 36938; 31938; 26938; 21938; 16938; 11938; 6938; 1938; 231938 raz


In [18]:
cre

,userId,score
0,61c087a98b7198000157ab67,61
1,61c08844e6086b0001a53da5,71
2,61c084e8e6086b0001a53c04,1
3,61c082dc8b7198000157a93f,25
4,61c07e86e6086b0001a5393b,1
...,...,...
143648,5e34c6d991c64d0001d54633,49
143649,5e34c25e91c64d0001d545a4,45
143650,5e34c008008e1d000103bfcd,32
143651,5e34c15791c64d0001d54589,15


In [19]:
cre = cre.rename(columns={"score": "odobrim_score"})
cre['KI'] = cre['odobrim_score'].apply(lambda x: "good" if x > 59 else "bad")

In [21]:
du2 = pd.merge(left=du[['dt', 'phone', 'firstName2', 'email', 'id', 'timezone',
                       'advertisingAgreementsSms', 'advertisingAgreementsEmail']], 
               right=cre[['userId', 'odobrim_score',# 'nbch', 'ecs', 'ei',
                          'KI'
                         ]].reset_index(drop=True), 
               left_on='id', 
               right_on='userId',
               how = 'left')
# du2 = pd.merge(left=du, 
#                right=cre.reset_index(drop=True), 
#                left_on='id', 
#                right_on='userId',
#                how = 'left')

In [22]:
du2['advertisingAgreementsSms'] = du2['advertisingAgreementsSms'].apply(lambda x: x[-1]['isAgree'] if 
                                                                        type(x)!=float and len(x) >0 
                                                                        else False)
du2['advertisingAgreementsEmail'] = du2['advertisingAgreementsEmail'].apply(lambda x: x[-1]['isAgree'] if 
                                                                        type(x)!=float and len(x) >0 
                                                                        else False)

In [23]:
du2.head()

,dt,phone,firstName2,email,id,timezone,advertisingAgreementsSms,advertisingAgreementsEmail,userId,odobrim_score,KI
0,2018-08-08 06:25:41.620,79999999999,Александр,fEdor@fepe.ru,5adf4162812abb00063d5db5,UTC+3:00,True,True,NaN,NaN,NaN
1,2018-08-08 00:00:00.000,79139223030,Наталья,nvinogradova@bk.ru,5ae19de7812abb00063d5dd2,UTC+7:00,True,True,NaN,NaN,NaN
2,2018-08-08 00:00:00.000,79150037080,Руслан,0037080@gmail.com,5ae2d649812abb00063d5dd4,UTC+3:00,True,True,5ae2d649812abb00063d5dd4,92.0,good
3,2018-08-08 00:00:00.000,79104017172,Илья,ChernyshevIA@msk.bcs.ru,5ae316e2812abb00063d5dd8,UTC+3:00,True,True,5ae316e2812abb00063d5dd8,55.0,bad
4,2018-08-08 00:00:00.000,79037303787,Олег,xlegus@gmail.com,5ae478061525640007977ffd,UTC+3:00,True,True,5ae478061525640007977ffd,40.0,bad


In [24]:
du3 = pd.merge(left=du2[du2['firstName2'].notnull()],#[['id', 'dt', 'phone','firstName2', 
#                                                       'email', 'KI', 'odobrim_score','salary','timezone',
#                                                      'advertisingAgreementsEmail','obl']],
         right = dff,
         left_on='id', 
         right_on='userId',
         how = 'left')
# du3 = pd.merge(left=du2[du2['firstName2'].notnull()],
#          right = dff,
#          left_on='id', 
#          right_on='userId',
#          how = 'left')

In [25]:
du3['KI'].fillna('NoKI', inplace=True)

In [26]:
len(du3)

177133

In [27]:
#Разобъем всех id по возрасту на 4 группы : 'до 30', '30-45', '45-60', '60+' 
def income_group(odobrim_score):
    if odobrim_score <= 49: return 'bad'
    if odobrim_score > 50: return 'good'
    


    else: 
        return 'NoKi'
    
du3['segment'] = du3['odobrim_score'].apply(income_group)
du3['segment'].value_counts()

bad     89124
NoKi    63450
good    24559
Name: segment, dtype: int64

In [28]:
# du3.groupby(['age_group','segment']).agg({'id_x':'count'})

KeyError: 'age_group'

In [29]:
# #Разобъем всех id по возрасту на 4 группы : 'до 30', '30-45', '45-60', '60+' 
# def income_group(odobrim_score):
#     if odobrim_score <= 20: return '0-20'
#     if 20 < odobrim_score <= 40: return '20-40'
#     if 40 < odobrim_score <= 60: return '40-60'
#     if 60 < odobrim_score <=80: return '60-80'
#     if 80 < odobrim_score <=100: return '80-100'


#     else: 
#         return '200000+'
    
# du3['income_group'] = du3['odobrim_score'].apply(income_group)
# du3['income_group'].value_counts()

In [30]:
# def get_kibana_reg(es, since,till):
#     gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
#     lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

#     payload = {"size": 1000,
# "query":{"bool":{"must":[{"match_all":{}},
#                          {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
#                          {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
#     d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
#     sleep(0.5)
#     doc_count = d['hits']['total']
#     hits = d['hits']['hits']
#     print(doc_count, end ="; ", flush=True)
#     res = []
#     for i in range(1, int(doc_count/1000)+1):
#         scroll = d['_scroll_id']
#         d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
#         hits = hits + d['hits']['hits']
#         print(doc_count-1000*i, end ="; ", flush=True)
#         sleep(0.5)

#     dr = pd.DataFrame([hit['_source'] for hit in hits])
#     dr.params.apply(lambda x: x.keys())
#     for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
#                 'yandexClientID','currentPage',
# #                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
# #                   'transaction_id', 'clickId', 
#                 'sourceGlobal']:
#         dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

#     dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
#     dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
#     return dr
# since = date(2018,1,1)
# till = date(2021,12,1)
# regs = get_kibana_reg(es, since,till)


In [31]:
# regs1=regs[~regs['productType'].isin(['loan','Займ'])]

# regs1 = regs1[['userId','productType']]
# regs1.columns = ['userId_x','productType']
# regs1

# merge = du3.merge(regs1,how='left',on='userId_x')
# len(merge)

# merge1=merge.drop_duplicates('userId_x')
# merge1.tail(4000)

# merge2 = merge1[merge1['age_group']!='24']
# len(merge2)

# merge2['KI2'] = merge2['KI']
# merge2['KI2'] = merge2['odobrim_score'].apply(lambda x: 'Not_Bad' if x==1 else x)
# # merge2['KI2'].unique()

# merge2[((merge2['KI']=='good'))|(merge2['KI2']=='Not_Bad')]

# df=merge2[merge2['age_group'].isin(['35-44','25-34','45-54'])]
# df = df[df['advertisingAgreementsEmail']==True]
# df = df[df['dt']>'2018-06-01']
# df = df[df['KI']=='good']
# #df.to_excel('broker_send.xlsx')

# df[['firstName2', 'email', 'id_x']].to_excel('broker_send.xlsx')

# merge1[merge1['dt']>'2021-05-01'].plot(x='dt',y='odobrim_score',figsize=(12,6))

# merge1['KI'].value_counts()

# len(merge1[merge1['odobrim_score']!='NoKI'])

# du3.drop_duplicates('userId_x').sum()

In [33]:
du3[((du3['odobrim_score']>49))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
                                                    'odobrim_score']].to_excel(f'reports/good.xlsx')
du3[((du3['odobrim_score']<49))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2',
                                                'email',
                                                'id_x', 
                                                'odobrim_score']].to_excel(f'reports/bad.xlsx')

In [ ]:
len(du3[((du3['odobrim_score']<49))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2',
                                                'email',
                                                'id_x', 
                                                'odobrim_score','age_group']])

In [ ]:
good_ = pd.read_excel(f'reports/good.xlsx')
bad_ = pd.read_excel(f'reports/bad.xlsx')


In [ ]:
bad1 = bad.sample(38007)

In [ ]:
bad1.to_excel('bad1.xlsx')

In [ ]:
baad=bad1['id_x'].to_list()

In [ ]:
baad

In [ ]:
bad2 = bad[~bad['id_x'].isin(baad)]
bad2.to_excel('bad2.xlsx')

In [ ]:
bad1

In [ ]:
du3[((du3['odobrim_score']>60))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
                                                    'odobrim_score','age_group']].to_excel(f'reports/good.xlsx')
du3[((du3['odobrim_score']<=60))&((du3['odobrim_score']>40))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2',
                                                'email',
                                                'id_x', 
                                                'odobrim_score','age_group']].to_excel(f'reports/avg.xlsx')
du3[((du3['odobrim_score']<=40))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2',
                                                'email',
                                                'id_x', 
                                                'odobrim_score','age_group']].to_excel(f'reports/bad.xlsx')


In [ ]:
good_ = pd.read_excel(f'reports/good.xlsx')
bad_ = pd.read_excel(f'reports/bad.xlsx')
merge = pd.concat([good_,bad_])
merge.to_excel('merge.xlsx')

In [ ]:
# bad_new = du3[&(du3['id_y'].isnull())&((du3['dt']>='2021-07-12'))
#    &(du3['dt']<datetime.now()-timedelta(days=3)) 
#    &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
#                                                     'odobrim_score','dt','age_group']].to_excel(f'reports/bad_new.xlsx') 

In [ ]:
# city_id = du3[((du3['odobrim_score']>=40))&(du3['obl']=='1')
#               &(du3['id_y'].isnull())&((du3['dt']>='2020-07-12'))
#    &(du3['dt']<datetime.now()-timedelta(days=3)) 
#    &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
#                                                     'odobrim_score','dt']].to_excel(f'reports/city_id.xlsx') 

In [ ]:
bad = pd.read_excel(f'reports/bad.xlsx')
len(bad)

In [ ]:
bad[bad['age_group'].isin(['24','25-34'])].to_excel('bad_24_25_34.xlsx')
bad[bad['age_group'].isin(['35-44','45-54'])].to_excel('bad_35_54.xlsx')
bad[bad['age_group'].isin(['65+'])].to_excel('bad_65.xlsx')
bad[bad['age_group'].isin(['NotBirthDay'])].to_excel('NotBirthDay.xlsx')



    
                          


In [ ]:
good = pd.read_excel(f'reports/good.xlsx')
good

In [ ]:
good[good['age_group'].isin(['24','25-34'])].to_excel('good_24_25_34.xlsx')
good[good['age_group'].isin(['35-44','45-54'])].to_excel('good_35_54.xlsx')
good[good['age_group'].isin(['65+'])].to_excel('good_65.xlsx')
good[good['age_group'].isin(['NotBirthDay'])].to_excel('good_NotBirthDay.xlsx')

In [ ]:
bad1 = bad.head(66985)
bad2 = bad.tail(66884)
bad1.to_excel('bad1.xlsx')
bad2.to_excel('bad2.xlsx')

In [ ]:
du3[((du3['odobrim_score']>=40))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
                                                    'odobrim_score']].to_excel(f'reports/middle.xlsx')

In [ ]:
du3[((du3['income_group']=='40-60'))&(du3['id_y'].isnull())
   &(du3['dt']<datetime.now()-timedelta(days=3)) 
   &(du3['advertisingAgreementsEmail']==True)][['firstName2',
                                                'email',
                                                'id_x', 
                                                'odobrim_score']].to_excel(f'reports/avg.xlsx')

In [ ]:
# du3[(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
#                                                      'odobrim_score']].to_excel('reports/everybody.xlsx')

In [ ]:
du3

In [ ]:
refin = pd.read_excel('рефин.xlsx')
mobi = pd.read_excel('мобилка.xlsx')
loyal = pd.read_excel('для_лояльност.xlsx')

In [ ]:
refin_phone = refin['phone'].to_list()

In [ ]:
mobi_phone = mobi['phone'].to_list()

In [ ]:
loyal_phone = loyal['phone'].to_list()

In [ ]:
refi_new = pd.read_excel('new_ref.xlsx')
refi_new

In [ ]:
merge_ref = refi_new.merge(du3, how='left',on='userId_x')

In [ ]:
merge_ref = merge_ref[['fullName','phone_y','timezone','dt']]
merge_ref=merge_ref.drop_duplicates('fullName')
merge_ref = merge_ref.sort_values('dt',ascending=False)
merge_ref_utc = merge_ref.head(200)
merge_ref_utc.to_excel('refin_utc.xlsx')

In [ ]:
merge_mobi = mobi.merge(du3, how='left',on='userId_x')

In [ ]:
merge_mobi = merge_mobi[['fullName','phone_y','timezone','dt']]
merge_mobi=merge_mobi.drop_duplicates('fullName')
merge_mobi = merge_mobi.sort_values('dt',ascending=False)
merge_mobi_utc = merge_mobi.head(200)
merge_mobi_utc.to_excel('mobi_utc.xlsx')
merge_mobi_utc

In [ ]:
merge_loyal = loyal.merge(du3, how='left',on='userId_x')

In [ ]:
merge_loyal = merge_loyal[['fullName','email','timezone','dt']]
merge_loyal=merge_loyal.drop_duplicates('fullName')
merge_loyal = merge_loyal.sort_values('dt',ascending=False)
merge_loyal = merge_loyal.head(200)
merge_loyal = merge_loyal[merge_loyal['phone_y'].notnull()]
merge_loyal.to_excel('loyal_utc.xlsx')
merge_loyal

In [ ]:
baza_mobi = du3[du3['phone'].isin(mobi)]
baza_mobi

In [ ]:
baza_loyal = du3[du3['phone'].isin(loyal)]
baza_loyal

In [ ]:
stas.drop_duplicates()

In [ ]:
stas = pd.read_excel('for_Stas.xlsx')
merge_stas = stas.merge(du3, how='left',on='userId_x')
merge_stas = merge_stas[['fullName','email','timezone','dt']]
merge_stas=merge_stas.drop_duplicates('fullName')
merge_stas = merge_stas.sort_values('dt',ascending=False)
merge_stas = merge_stas[merge_stas['email'].notnull()]
merge_stas.to_excel('pledge_auto.xlsx')
len(merge_stas)

In [ ]:
email=merge_stas['email'].to_list()

In [ ]:
stas_kn = pd.read_excel('for_Stas_kn.xlsx')

In [ ]:
stas_kn_fin=stas_kn.merge(du3, how='left',on='userId_x')
stas_kn_fin=stas_kn_fin[stas_kn_fin['KI']=='good']
stas_kn_fin

In [ ]:
names = pd.read_csv('russian_names.csv',sep=';')
names.head()

In [ ]:
names = names[['Name','Sex']]
names = names.rename(columns={
                        "Name": "firstName2"})

In [ ]:
stas_kn_fin = stas_kn_fin.merge(names,how='left',on='firstName2')
stas_kn_fin.head()

In [ ]:
stas_kn_fin_m=stas_kn_fin[stas_kn_fin['Sex']=='М']
stas_kn_fin_m = stas_kn_fin_m.sort_values('dt_x',ascending=False)
stas_kn_fin_m = stas_kn_fin_m.drop_duplicates('phone_x')
stas_kn_fin_m=stas_kn_fin_m[['fullName','email']]
stas_kn_fin_m=stas_kn_fin_m[~stas_kn_fin_m['email'].isin(email)]
len(stas_kn_fin_m)
stas_kn_fin_m.to_excel('kn_good_man.xlsx')

In [ ]:
stas_kn_fin_w=stas_kn_fin[stas_kn_fin['Sex']=='Ж']
stas_kn_fin_w = stas_kn_fin_w.sort_values('dt_x',ascending=False)
stas_kn_fin_w = stas_kn_fin_w.drop_duplicates('phone_x')
stas_kn_fin_w=stas_kn_fin_w[['fullName','email']]
stas_kn_fin_w=stas_kn_fin_w[~stas_kn_fin_w['email'].isin(email)]
stas_kn_fin_w.to_excel('kn_good_w.xlsx')

In [ ]:
stas_kn_fin_m[stas_kn_fin_m['email'].isin(email)]

In [ ]:
stas_kn_fin_w[stas_kn_fin_w['email'].isin(email)]

In [ ]:
# man=du3[((du3['odobrim_score']<56))&(du3['id_y'].isnull())
#    &(du3['dt']<datetime.now()-timedelta(days=3))& (du3['Sex']=='М') & (du3['salary']<'61000')
#    &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
#                                                     'odobrim_score','salary','Sex']]
# woman=du3[((du3['odobrim_score']<56))&(du3['id_y'].isnull())
#    &(du3['dt']<datetime.now()-timedelta(days=3))& (du3['Sex']=='Ж')&(du3['salary']<'61000') 
#    &(du3['advertisingAgreementsEmail']==True)][['firstName2', 'email', 'id_x', 
#                                                     'odobrim_score','salary','Sex']]

In [ ]:
# man = man[man['odobrim_score']>30]
# woman = woman[woman['odobrim_score']>30]

In [ ]:
# man.to_excel(f'reports/M.xlsx')
# woman.to_excel(f'reports/Ж.xlsx')

In [ ]:
bad = pd.read_excel('домен гуд.xlsx')

In [ ]:
bad['домен'].value_counts()

In [ ]:
good = pd.read_excel('домен бед.xlsx')

In [ ]:
good['домен'].value_counts()

In [ ]:
df = pd.read_excel('домен бед.xlsx')

In [ ]:
df.groupby('домен').agg({'id_x':'count'}).sort_values('id_x',ascending = False).head(100)